In [1]:
def add_subtitles(video_path, new_video_path, subtitles, font_size, line_thickness, color, bg_color, background_transparency=0.5):
    """Adds subtitles to a video file.

    Args:
        video_path: The path to the input video file.
        new_video_path: The path to the ouput video file.
        subtitles: A list of subtitles, each of which is a tuple of (start_time, end_time, text).
        
        optional config variables:
        font_size: size of font.
        line_thickness: line height.
        color = text color (r,g,b).
        bg_color = background color (r,g,b), None for transparent.


    Returns:
        The path to the new video file with the subtitles added.
    """

    import cv2
    import numpy as np

    video = cv2.VideoCapture(video_path)
    width = int(video.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(video.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    output_video = cv2.VideoWriter(new_video_path , fourcc, video.get(cv2.CAP_PROP_FPS), (width, height))

    for start_time, end_time, text in subtitles:
        start_frame = int(start_time * video.get(cv2.CAP_PROP_FPS))
        end_frame = int(end_time * video.get(cv2.CAP_PROP_FPS))

        for frame_num in range(0, int(video.get(cv2.CAP_PROP_FRAME_COUNT))):
            success, frame = video.read()
            if not success:
                break

            if frame_num >= start_frame and frame_num <= end_frame:
                font = cv2.FONT_HERSHEY_SIMPLEX

                
                # Create a copy of the frame to draw the text and background
                frame_with_text = frame.copy()

                # Get the size of the text to calculate the background rectangle dimensions
                text_width, text_height  = cv2.getTextSize(text, font, font_size, line_thickness)[0]

                # Calculate background rectangle dimensions
                bg_width = text_width + 20  # Adding some padding
                bg_height = text_height + 20

                bg_p1 = (int((width-bg_width)/2), height - 20 - bg_height)
                bg_p2 = (int((width+bg_width)/2), height - 20)

                # Calculate the position to center the text within the background rectangle
                text_x = bg_p1[0] + 10
                text_y = bg_p1[1] + 10 + text_height

                # Draw the background rectangle
                if bg_color is not None:
                    cv2.rectangle(frame_with_text, bg_p1, bg_p2, bg_color, -1)
                    # Combine the frame with the text and the original frame
                    frame = cv2.addWeighted(frame, 1-background_transparency, frame_with_text, background_transparency, 0)

                # Draw the centered text on top of the background rectangle
                cv2.putText(frame, text, (text_x, text_y), font, font_size, color, line_thickness, cv2.LINE_AA)


            output_video.write(frame)

    video.release()
    output_video.release()

In [2]:
add_subtitles('user_data/videos/testing.mp4', 'user_data/generated_videos/testing.mp4', [(1, 20, "hello")], 1, 1, (0,0,0), (255,255,255), 1)